In [8]:
import requests
import json
import os
import climate
from pprint import pprint
import pandas as pd
import numpy as np
import io
import census
from pprint import pprint

In [9]:
cd = census.CensusData()
ca = climate.ClimateAgent()
pgpw = os.getenv("POSTGRES_PASSWORD") 
server, engine = cd.connect_to_postgres(password=pgpw) #need to add create=True if


In [15]:
id = ca.get_station_id_from_postgres("norfolk", engine=engine)
ca.get_climate_data_from_postgres(station_id=id, engine=engine)

ProgrammingError: (psycopg.errors.UndefinedColumn) column "id" does not exist
LINE 4:         WHERE id='USW00013737'
                      ^
[SQL: 
        SELECT * 
        FROM climate_data
        WHERE id='USW00013737'
        ]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [ ]:
query = """
SELECT *
FROM city_dataa
"""

pd.read_sql_query(query, con=engine)

,Unnamed: 0,city,year,male_worker_earnings,female_worker_earnings,pct_below_pov,num_housing_units,num_married_housing_units,avg_household_size,lt_10k_income,...,pct_in_retail,pct_in_transportation,pct_in_information,pct_in_finance,pct_in_science,pct_in_education,pct_in_arts,pct_in_pub_ad,state,county
0,0,albemarle,2009,32296,51355,(X),41613,23837,3.05,36295,...,1.8,8.9,3.5,2.3,6.3,12.3,33.4,4.9,51,3
1,1,albemarle,2010,53850,41920,5.5,42175,24377,3.14,2512,...,11.4,2.4,2.2,6.9,14.4,35.7,6.7,2.8,51,3
2,2,albemarle,2011,55530,52211,3.8,42468,22450,3.23,2692,...,7.4,2.1,1.8,5.0,14.0,37.5,10.5,5.3,51,3
3,3,albemarle,2012,59828,39656,6.1,43302,26335,2.96,1757,...,8.1,2.5,2.1,5.1,12.9,37.0,8.0,5.3,51,3
4,4,albemarle,2013,52453,46615,8.2,43483,25853,2.92,2452,...,7.1,2.3,1.0,7.0,12.1,36.6,9.8,6.1,51,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,413,beach,2018,55187,43040,4.5,186134,116678,3.07,5589,...,11.5,4.2,1.6,8.0,13.1,22.6,10.2,9.5,51,810
414,414,beach,2019,54513,45324,4.4,186468,89905,2.52,5411,...,10.5,4.7,1.2,7.6,13.6,25.0,11.4,7.6,51,810
415,415,beach,2021,61234,50995,7.5,190981,88104,2.45,7841,...,10.8,4.6,1.8,6.8,15.2,23.3,11.4,8.2,51,810
416,416,beach,2022,62259,54102,7.7,191169,84113,2.49,8962,...,11.8,3.4,2.2,9.6,14.6,22.8,9.0,8.2,51,810


In [19]:
# to delete a table,,, weird syntax
from sqlalchemy import MetaData, Table
metadata = MetaData()

# Reflect the existing table
table_name = 'climate_data'
table = Table(table_name, metadata, autoload_with=engine)

# Drop the table
table.drop(engine)

In [9]:
cities_and_ids = pd.read_csv("data/city-ids.csv")
cities = list(cities_and_ids['city'])
station_ids = list(cities_and_ids['id'])
cd.upload_cities_to_postgres(cities_and_ids, engine=engine)

Uploading region information to database
Finished uploading to database


In [11]:
y = "2022"
tmp_url = cd.base_url + f"{y}/acs/acs1/profile"
get_metrics = "".join(x + "," for x in cd.list_of_metrics.keys())
get_metrics = get_metrics[:len(get_metrics)-1] # get rid of the last comma that I appended
city_id = "003"
           
params = {
        "get": get_metrics,
        "for": f"county:{city_id}",
        "in": "state:51",
            "key": os.getenv("CENSUS_KEY")}

r = requests.get(tmp_url, params=params)
r.url

'https://api.census.gov/data/2022/acs/acs1/profile?get=NAME%2CDP03_0093E%2CDP03_0094E%2CDP03_0119PE%2CDP04_0001E%2CDP02_0002E%2CDP02_0016E%2CDP03_0052E%2CDP03_0053E%2CDP03_0054E%2CDP03_0055E%2CDP03_0056E%2CDP03_0057E%2CDP03_0058E%2CDP03_0059E%2CDP03_0060E%2CDP03_0061E%2CDP02_0026E%2CDP02_0028E%2CDP02_0030E%2CDP02_0054E%2CDP02_0055E%2CDP02_0056E%2CDP02_0057E%2CDP02_0058E%2CDP02_0060PE%2CDP02_0061PE%2CDP02_0062PE%2CDP02_0063PE%2CDP02_0064PE%2CDP02_0065PE%2CDP02_0066PE%2CDP03_0009PE%2CDP03_0033PE%2CDP03_0034PE%2CDP03_0035PE%2CDP03_0036PE%2CDP03_0037PE%2CDP03_0038PE%2CDP03_0039PE%2CDP03_0040PE%2CDP03_0041PE%2CDP03_0042PE%2CDP03_0043PE%2CDP03_0045PE&for=county%3A003&in=state%3A51&key=9602e5c45b622cac9d54ce80349a8db0fd85101e'

In [4]:
ids = list(cd.get_all_cities()['county'])
full_df = pd.DataFrame()
for id in ids:
    df = cd.get_data_for_city(city="blank", years=[2009, 2024], county=id)
    full_df = pd.concat([full_df, df], ignore_index=True)

full_df

starting scraping for: 003
r <Response [404]>
finished scraping for: 003
starting scraping for: 013
r <Response [404]>
finished scraping for: 013
starting scraping for: 015
r <Response [404]>
finished scraping for: 015
starting scraping for: 019
r <Response [404]>
finished scraping for: 019
starting scraping for: 041
r <Response [404]>
finished scraping for: 041
starting scraping for: 059
r <Response [404]>
finished scraping for: 059
starting scraping for: 061
r <Response [404]>
finished scraping for: 061
starting scraping for: 069
r <Response [404]>
finished scraping for: 069
starting scraping for: 085
r <Response [404]>
finished scraping for: 085
starting scraping for: 087
r <Response [404]>
finished scraping for: 087
starting scraping for: 095
r <Response [204]>
r <Response [404]>
finished scraping for: 095
starting scraping for: 107
r <Response [404]>
finished scraping for: 107
starting scraping for: 121
r <Response [404]>
finished scraping for: 121
starting scraping for: 153
r <Re

,city,year,male_worker_earnings,female_worker_earnings,pct_below_pov,num_housing_units,num_married_housing_units,avg_household_size,lt_10k_income,gt_10k_lt_15k_income,...,pct_in_retail,pct_in_transportation,pct_in_information,pct_in_finance,pct_in_science,pct_in_education,pct_in_arts,pct_in_pub_ad,state,county
0,albemarle,2009,32296,51355,(X),41613,23837,3.05,36295,2519,...,1.8,8.9,3.5,2.3,6.3,12.3,33.4,4.9,51,003
1,albemarle,2010,53850,41920,5.5,42175,24377,3.14,2512,1589,...,11.4,2.4,2.2,6.9,14.4,35.7,6.7,2.8,51,003
2,albemarle,2011,55530,52211,3.8,42468,22450,3.23,2692,1915,...,7.4,2.1,1.8,5.0,14.0,37.5,10.5,5.3,51,003
3,albemarle,2012,59828,39656,6.1,43302,26335,2.96,1757,1403,...,8.1,2.5,2.1,5.1,12.9,37.0,8.0,5.3,51,003
4,albemarle,2013,52453,46615,8.2,43483,25853,2.92,2452,977,...,7.1,2.3,1.0,7.0,12.1,36.6,9.8,6.1,51,003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,beach,2018,55187,43040,4.5,186134,116678,3.07,5589,4035,...,11.5,4.2,1.6,8.0,13.1,22.6,10.2,9.5,51,810
414,beach,2019,54513,45324,4.4,186468,89905,2.52,5411,2120,...,10.5,4.7,1.2,7.6,13.6,25.0,11.4,7.6,51,810
415,beach,2021,61234,50995,7.5,190981,88104,2.45,7841,5502,...,10.8,4.6,1.8,6.8,15.2,23.3,11.4,8.2,51,810
416,beach,2022,62259,54102,7.7,191169,84113,2.49,8962,4238,...,11.8,3.4,2.2,9.6,14.6,22.8,9.0,8.2,51,810


In [4]:
cities_to_get = list(pd.read_csv("data/city-data.csv")['city'].unique())
query = """
SELECT *
FROM city_ids
"""

city_ids_df = pd.read_sql_query(query, con=engine).drop("Unnamed: 0", axis=1)

climate_data = pd.DataFrame()
for c in cities_to_get:
    print('working on:', c)
    if c == "fauquier":
        c = "front royal"
    if c == "jamescity":
        c = "williamsburg"
    if c == "princewilliam":
        c = "manassas"
    if c == "spotsylvania":
        c = "fredericksburg"
    if c == "albemarle":
        c = "charlottesville"
    if c == "montgomery":
        continue

    if c == "beach":
        c = "Virginia Beach"
    id = city_ids_df[city_ids_df['city'].str.contains(c)]['id'].values

    if len(id) > 0:
        print(id[0])
        try:
            df = ca.get_data_for_station_id(id[0], start_date="1970-01-01", end_date="2023-01-01")
        except Exception as e:
            print(e)
            continue
        climate_data = pd.concat([climate_data, df], ignore_index=True)

climate_data
    
    

working on: albemarle
USW00093736
           STATION        DATE  PRCP  RHAV  TMAX  TMIN  TSUN
0      USW00093736  1970-01-01  0.00   NaN  35.0  30.0   NaN
1      USW00093736  1970-01-02  0.00   NaN  40.0  25.0   NaN
2      USW00093736  1970-01-03  0.00   NaN  41.0  25.0   NaN
3      USW00093736  1970-01-04  0.00   NaN  35.0  19.0   NaN
4      USW00093736  1970-01-05  0.00   NaN  44.0  19.0   NaN
...            ...         ...   ...   ...   ...   ...   ...
19343  USW00093736  2022-12-28  0.00   NaN  54.0  22.0   NaN
19344  USW00093736  2022-12-29  0.00   NaN  65.0  30.0   NaN
19345  USW00093736  2022-12-30  0.00   NaN  69.0  35.0   NaN
19346  USW00093736  2022-12-31  0.25   NaN  59.0  51.0   NaN
19347  USW00093736  2023-01-01  0.00   NaN  69.0  46.0   NaN

[19348 rows x 7 columns]
working on: arlington
USW00013743
           STATION        DATE  PRCP  RHAV  TMAX  TMIN   TSUN
0      USW00013743  1970-01-01  0.00   NaN    35    24    0.0
1      USW00013743  1970-01-02  0.00   NaN    41  

,STATION,DATE,PRCP,RHAV,TMAX,TMIN,TSUN
0,USW00093736,1970-01-01,0.00,NaN,35.0,30.0,NaN
1,USW00093736,1970-01-02,0.00,NaN,40.0,25.0,NaN
2,USW00093736,1970-01-03,0.00,NaN,41.0,25.0,NaN
3,USW00093736,1970-01-04,0.00,NaN,35.0,19.0,NaN
4,USW00093736,1970-01-05,0.00,NaN,44.0,19.0,NaN
...,...,...,...,...,...,...,...
425143,USW00013769,2022-12-28,0.00,NaN,54.0,24.0,NaN
425144,USW00013769,2022-12-29,0.00,NaN,62.0,26.0,NaN
425145,USW00013769,2022-12-30,0.00,NaN,67.0,34.0,NaN
425146,USW00013769,2022-12-31,0.98,NaN,67.0,54.0,NaN


In [3]:
id = "USW00013769"
ca.get_data_for_station_id(id, start_date="1970-01-01", end_date="2023-01-01")

,STATION,DATE,PRCP,RHAV,TMAX,TMIN,TSUN
0,USW00013769,1970-01-01,0.03,NaN,40.0,36.0,NaN
1,USW00013769,1970-01-02,0.00,NaN,41.0,29.0,NaN
2,USW00013769,1970-01-03,0.00,NaN,44.0,30.0,NaN
3,USW00013769,1970-01-04,0.00,NaN,36.0,22.0,NaN
4,USW00013769,1970-01-05,0.00,NaN,44.0,21.0,NaN
...,...,...,...,...,...,...,...
18139,USW00013769,2022-12-28,0.00,NaN,54.0,24.0,NaN
18140,USW00013769,2022-12-29,0.00,NaN,62.0,26.0,NaN
18141,USW00013769,2022-12-30,0.00,NaN,67.0,34.0,NaN
18142,USW00013769,2022-12-31,0.98,NaN,67.0,54.0,NaN


In [5]:
climate_data.to_csv("data/climate-data.csv")

In [ ]:
cities_and_ids = pd.read_csv("data/city-ids.csv")
cities = list(cities_and_ids['city'])
station_ids = list(cities_and_ids['id'])
cd.upload_cities_to_postgres(cities_and_ids, engine=engine)

print("Getting all city data")
counties = list(cities_and_ids['id'])
city_df = pd.DataFrame()
for c in cities:
    print("Working on", c)
    try:
        if c == "fauquier":
            c = "front royal"
        if c == "jamescity":
            c = "williamsburg"
        if c == "princewilliam":
            c = "manassas"
        if c == "spotsylvania":
            c = "fredericksburg"
        if c == "albemarle":
            c = "charlottesville"

        if c == "beach":
            c = "Virginia Beach"
        df = cd.get_data_for_city(city=c, years=[2009, 2023])
        city_df = pd.concat([city_df, df], ignore_index=True)
    except Exception as e:
        print(e)
        continue
print(city_df)
cd.upload_city_data_to_postgres(city_df, engine=engine)

Uploading region information to database
Finished uploading to database
Getting all city data
Working on charlottesville
URL ERROR, MAKE SURE IT IS CORRECTLY CONFIGURED
Working on arlington
starting scraping for: 013
r <Response [404]>
finished scraping for: 013
Working on augusta
starting scraping for: 015
r <Response [200]>
r <Response [404]>
finished scraping for: 015
Working on bedford
starting scraping for: 019


In [5]:
query = """
SELECT * from city_data
"""

pd.read_sql_query(query, con=engine)

,city,year,male_worker_earnings,female_worker_earnings,pct_below_pov,num_housing_units,num_married_housing_units,avg_household_size,lt_10k_income,gt_10k_lt_15k_income,...,pct_in_retail,pct_in_transportation,pct_in_information,pct_in_finance,pct_in_science,pct_in_education,pct_in_arts,pct_in_pub_ad,state,county
0,albemarle,2009,32296,51355,(X),41613,23837,3.05,36295,2519,...,1.8,8.9,3.5,2.3,6.3,12.3,33.4,4.9,51,003
1,albemarle,2010,53850,41920,5.5,42175,24377,3.14,2512,1589,...,11.4,2.4,2.2,6.9,14.4,35.7,6.7,2.8,51,003
2,albemarle,2011,55530,52211,3.8,42468,22450,3.23,2692,1915,...,7.4,2.1,1.8,5.0,14.0,37.5,10.5,5.3,51,003
3,albemarle,2012,59828,39656,6.1,43302,26335,2.96,1757,1403,...,8.1,2.5,2.1,5.1,12.9,37.0,8.0,5.3,51,003
4,albemarle,2013,52453,46615,8.2,43483,25853,2.92,2452,977,...,7.1,2.3,1.0,7.0,12.1,36.6,9.8,6.1,51,003
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
413,beach,2018,55187,43040,4.5,186134,116678,3.07,5589,4035,...,11.5,4.2,1.6,8.0,13.1,22.6,10.2,9.5,51,810
414,beach,2019,54513,45324,4.4,186468,89905,2.52,5411,2120,...,10.5,4.7,1.2,7.6,13.6,25.0,11.4,7.6,51,810
415,beach,2021,61234,50995,7.5,190981,88104,2.45,7841,5502,...,10.8,4.6,1.8,6.8,15.2,23.3,11.4,8.2,51,810
416,beach,2022,62259,54102,7.7,191169,84113,2.49,8962,4238,...,11.8,3.4,2.2,9.6,14.6,22.8,9.0,8.2,51,810


In [ ]:
cities = list(cd.get_all_cities()['city'])
print(cities)

cities_and_ids = {"city": [],
                  "id": []}
print("Getting all city names and ids")
for c in cities:

    if c == "fauquier":
        c = "front royal"
    if c == "jamescity":
        c = "williamsburg"
    if c == "princewilliam":
        c = "manassas"
    if c == "spotsylvania":
        c = "fredericksburg"
    if c == "albemarle":
        c = "charlottesville"

    if c == "beach":
        c = "Virginia Beach"
    
    c = c + ", virginia"
    id = ca.get_station_id(c)
    cities_and_ids['city'].append(c)
    cities_and_ids['id'].append(id)



['albemarle', 'arlington', 'augusta', 'bedford', 'chesterfield', 'fairfax', 'fauquier', 'frederick', 'hanover', 'henrico', 'jamescity', 'loudoun', 'montgomery', 'princewilliam', 'roanoke', 'rockingham', 'spotsylvania', 'stafford', 'york', 'lynchburg', 'alexandria', 'chesapeake', 'hampton', 'newportnews', 'norfolk', 'portsmouth', 'richmond', 'roanoke', 'suffolk', 'beach']
Getting all city names and ids


In [11]:
city_ids = {
    "charlottesville": "USW00093736", 
    "arlington": "USW00013743", 
    "augusta": "USC00448062",
    "bedford": "USC00440551",
    "chesterfield": "USW00000153",
    "fairfax": "USW00093738",
    "front royal": "USC00443229",
    "frederick": "USC00449186",
    "hanover": "USW00093775",
    "henrico": "USW00013740",
    "williamsburg": "USC00449151",
    "loudon": "USC00444909",
    "montgomery": "USC00440766",
    "manassas": "USC00445204",
    "roanoke": "USW00013741",
    "rockingham": "USC00442208",
    "fredericksburg": "USC00442009",
    "stafford": "USW00013773",
    "york": "USW00093741",
    "lynchburg": "USW00013733",
    "alexandria": "USW00013743",
    "chesapeake": "USW00013762",
    "hampton": "USW00013737",
    "newportnews": "USW00093741",
    "norfolk": "USW00013737",
    "portsmouth": "USW00013737",
    "richmond": "USW00013740",
    "roanoke": "USW00013741",
    "suffolk": "USC00448192",
    "Virginia Beach": "USW00013769"
    }

new_dict = {"city":[], "id": []}

for key, value in city_ids.items():
    new_dict['city'].append(key)
    new_dict['id'].append(value)



city_ids = pd.DataFrame.from_dict(new_dict)

city_ids.to_csv("data/city-ids.csv")

In [ ]:
# cd.upload_cities_to_postgres(cities_and_ids, engine=engine)
#cities_and_ids.to_sql("city_ids", con=engine, index=False, chunksize=1000, if_exists="append")


    
final_df = pd.DataFrame.from_dict(cities_and_ids)
final_df

,city,id
0,"charlottesville, virginia",USW00093736
1,"arlington, virginia",USW00013743
2,"augusta, virginia",USC00448062
3,"bedford, virginia",USC00440551
4,"chesterfield, virginia",USW00000153
5,"fairfax, virginia",USW00093738
6,"front royal, virginia",USC00443229
7,"frederick, virginia",USC00449186
8,"hanover, virginia",USW00093775
9,"henrico, virginia",USW00013740


In [12]:
query = """
SELECT *
FROM city_ids
"""

pd.read_sql(query, con=engine)

NameError: name 'engine' is not defined

In [ ]:
cities = list(cd.get_all_cities()['city'])
print(cities)

# cities_and_ids = pd.DataFrame(columns=["city", "id"])
# print("Getting all city names and ids")
# for c in cities:
#     try:
#         if c == "fauquier":
#             c = "front royal"
#         if c == "jamescity":
#             c = "williamsburg"
#         if c == "princewilliam":
#             c = "manassas"
#         if c == "spotsylvania":
#             c = "fredericksburg"

#         if c == "beach":
#             c = "Virginia Beach"
#         print(c)
#         id = ca.get_station_id(c)
#         df = pd.DataFrame({c:id}, columns=['city', 'id'])
#         cities_and_ids = pd.concat([cities_and_ids, df], ignore_index=True)

#     except:
#         continue

# print("Getting all city data")
# city_df = pd.DataFrame()
# for c in cities:
#     try:
#         df = cd.get_data_for_city(c)
#         city_df = pd.concat([city_df, df], ignore_index=True)
#     except:
#         continue

# cd.upload_city_data_to_postgres(city_df, engine=engine)

print("Getting all climate data")
full_df = pd.DataFrame()
for c in cities:
    try:
        if c == "fauquier":
            c = "front royal"
        if c == "jamescity":
            c = "williamsburg"
        if c == "princewilliam":
            c = "manassas"
        if c == "spotsylvania":
            c = "fredericksburg"

        if c == "beach":
            c = "Virginia Beach"
        c = c + ", virginia"
        print("Working on:", c)
        station_id = ca.get_station_id(c)
        climate_df = ca.get_data_for_station_id(station_id=station_id, start_date="1940-01-01", end_date="2023-12-31")
        full_df = pd.concat([full_df, climate_df], ignore_index=True)
    except:
        continue

ca.upload_data_to_postgres(full_df, engine=engine)

['albemarle', 'arlington', 'augusta', 'bedford', 'chesterfield', 'fairfax', 'fauquier', 'frederick', 'hanover', 'henrico', 'jamescity', 'loudoun', 'montgomery', 'princewilliam', 'roanoke', 'rockingham', 'spotsylvania', 'stafford', 'york', 'lynchburg', 'alexandria', 'chesapeake', 'hampton', 'newportnews', 'norfolk', 'portsmouth', 'richmond', 'roanoke', 'suffolk', 'beach']
Getting all city names and ids
albemarle
arlington
augusta
bedford
chesterfield
fairfax
front royal
frederick
hanover
henrico
williamsburg
loudoun
montgomery
manassas


In [8]:
query = """
SELECT *
FROM city_ids
"""

pd.read_sql(query, con=engine)

,city,id


In [110]:
import plotly.express as px
import matplotlib as plt
query = """
SELECT *
FROM city_ids
"""
df = pd.read_sql_query(query, con=engine)
df

,city,id
0,"albemarle, virginia",USW00093736
1,"arlington, virginia",USW00013743
2,"augusta, virginia",USC00448062
3,"bedford, virginia",USC00440551
4,"chesterfield, virginia",USW00000153
5,"fairfax, virginia",USW00093738
6,"front royal, virginia",USC00443229
7,"frederick, virginia",USC00449186
8,"hanover, virginia",USW00093775
9,"henrico, virginia",USW00013740


In [ ]:
station_id_one = cli.get_station_id("norfolk, virginia")
station_id_two = cli.get_station_id("augusta, virginia")

df_one = cli.get_climate_data_from_postgres(station_id_one, engine=engine)
df_two = cli.get_climate_data_from_postgres(station_id_two, engine=engine)

full_df = pd.concat([df_one, df_two])
full_df['date'] = pd.to_datetime(full_df['date'])
full_df['year'] = full_df['date'].dt.year
full_df['month'] = full_df['date'].dt.month
full_df.groupby(["id", "year"]).agg(
    {"prcp": "mean",
     "tmax": "mean",
     "tmin": "mean"}
)




prcp       tmax       tmin
id          year                                
USC00448062 1940  0.125246  64.136986  40.571038
            1941  0.063003  68.254190  40.819444
            1942  0.126686  66.779614  41.532967
            1943  0.091151  67.672176  41.791781
            1944  0.113297  67.964286  42.691257
...                    ...        ...        ...
USW00013737 2019  0.126521  71.594521  55.191781
            2020  0.153907  72.139344  55.871585
            2021  0.106630  70.717808  53.983562
            2022  0.107425  69.764384  52.906849
            2023  0.138493  70.865753  54.610959

[162 rows x 3 columns]

In [ ]:
headers = cli.make_headers()
station = cli.get_station_id("norfolk, virginia")
params = {"dataset":"global-summary-of-the-month",
          "stations": station,
          "startDate": "2022-01-01",
          "endDate": "2022-01-02"
          }
r = requests.get("https://www.ncei.noaa.gov/access/services/search/v1/data/", params=params, headers=headers)
data_types = json.loads(r.text)['results'][0]['dataTypes']
data_types    

In [63]:
query = """
SELECT *
FROM climate_data
LIMIT 1
"""
df = pd.read_sql_query(query, con=engine)
cols = list(df.columns)

for c in cols:
    print(c)

id
date
prcp
tmax
tmin


In [58]:
query = f"""
SELECT *
FROM city_data
WHERE station == '12345'
"""
record = pd.read_sql_query(query, con=engine)

ProgrammingError: (psycopg.errors.UndefinedColumn) column "station" does not exist
LINE 4: WHERE station == '12345'
              ^
HINT:  Perhaps you meant to reference the column "city_data.state".
[SQL: 
SELECT *
FROM city_data
WHERE station == '12345'
]
(Background on this error at: https://sqlalche.me/e/20/f405)

In [38]:
get_metrics = "".join(x + "," for x in cd.list_of_metrics.keys())
get_metrics = get_metrics[:len(get_metrics)-1] # get rid of the last comma that I appended
headers = ca
y=2019

city_id = cd.find_city_id("norfolk")

tmp_url = cd.base_url + f"{y}/acs/acs1/profile"

           
params = {
    "get": get_metrics,
    "for": f"county:{city_id}",
    "in": "state:51",
    "key": os.getenv("CENSUS_KEY")}

r = requests.get(tmp_url, params)
json.loads(r.text)

[['NAME',
  'DP03_0093E',
  'DP03_0094E',
  'DP03_0119PE',
  'DP04_0001E',
  'DP02_0002E',
  'DP02_0016E',
  'DP03_0052E',
  'DP03_0053E',
  'DP03_0054E',
  'DP03_0055E',
  'DP03_0056E',
  'DP03_0057E',
  'DP03_0058E',
  'DP03_0059E',
  'DP03_0060E',
  'DP03_0061E',
  'DP02_0026E',
  'DP02_0028E',
  'DP02_0030E',
  'DP02_0054E',
  'DP02_0055E',
  'DP02_0056E',
  'DP02_0057E',
  'DP02_0058E',
  'DP02_0060PE',
  'DP02_0061PE',
  'DP02_0062PE',
  'DP02_0063PE',
  'DP02_0064PE',
  'DP02_0065PE',
  'DP02_0066PE',
  'DP03_0009PE',
  'DP03_0033PE',
  'DP03_0034PE',
  'DP03_0035PE',
  'DP03_0036PE',
  'DP03_0037PE',
  'DP03_0038PE',
  'DP03_0039PE',
  'DP03_0040PE',
  'DP03_0041PE',
  'DP03_0042PE',
  'DP03_0043PE',
  'DP03_0045PE',
  'state',
  'county'],
 ['Norfolk city, Virginia',
  '40180',
  '35197',
  '12.7',
  '98476',
  '29431',
  '2.45',
  '8092',
  '4136',
  '8073',
  '10024',
  '10885',
  '17685',
  '11507',
  '10279',
  '4615',
  '3091',
  '56844',
  '1868',
  '10295',
  '4199',
  

In [22]:
# to delete a table,,, weird syntax
from sqlalchemy import MetaData, Table
metadata = MetaData()

# Reflect the existing table
table_name = 'city_ids'
table = Table(table_name, metadata, autoload_with=engine)

# Drop the table
table.drop(engine)

In [14]:
query = """
SELECT * 
FROM city_data
"""

pd.read_sql_query(query, con=engine)

,city,year,male_worker_earnings,female_worker_earnings,pct_below_pov,num_housing_units,num_married_housing_units,avg_household_size,lt_10k_income,gt_10k_lt_15k_income,...,pct_in_retail,pct_in_transportation,pct_in_information,pct_in_finance,pct_in_science,pct_in_education,pct_in_arts,pct_in_pub_ad,state,county
0,arlington,2009,53566,71361,(X),103813,38713,3.39,90561,4313,...,0.4,5.2,1.6,3.8,9.1,26.9,13.2,6.6,51,013
1,arlington,2010,73658,62133,6.1,105490,42683,3.01,3126,1527,...,6.1,1.9,2.8,5.8,24.5,17.8,7.6,17.8,51,013
2,arlington,2011,81465,62031,4.6,106720,41499,3.23,2915,1426,...,3.6,1.2,2.8,8.1,29.2,17.9,6.8,19.3,51,013
3,arlington,2012,87362,65904,6.2,107734,42934,3.12,3033,1914,...,6.1,1.9,2.9,8.3,29.8,14.4,6.8,16.4,51,013
4,arlington,2013,89109,75891,8.2,109689,48136,3.00,4281,1830,...,3.8,1.4,3.5,7.0,29.5,14.7,10.0,15.9,51,013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
267,suffolk,2017,54963,42263,9.4,36845,23708,3.30,2696,1311,...,9.6,7.8,1.4,5.2,9.9,26.8,7.7,9.3,51,800
268,suffolk,2018,61750,46675,6.4,37333,24434,3.15,1585,1220,...,9.1,7.4,1.0,5.1,12.1,24.9,6.5,9.3,51,800
269,suffolk,2019,60358,42672,7.0,37954,17742,2.65,1570,818,...,11.8,8.4,2.7,5.6,9.3,21.5,8.4,11.1,51,800
270,suffolk,2021,62276,47303,9.2,39558,18713,2.55,1903,824,...,7.8,8.7,1.3,4.7,11.5,25.9,7.6,6.1,51,800


Uploading region information to database
Finished uploading to database
Getting all city data
https://api.census.gov/data/2022/acs/acs1/profile?key=9602e5c45b622cac9d54ce80349a8db0fd85101e&get=NAME&for=county%3A%2A&in=state%3A51
URL ERROR, MAKE SURE IT IS CORRECTLY CONFIGURED
'str' object cannot be interpreted as an integer
'str' object cannot be interpreted as an integer
'str' object cannot be interpreted as an integer
'str' object cannot be interpreted as an integer
'str' object cannot be interpreted as an integer
https://api.census.gov/data/2022/acs/acs1/profile?key=9602e5c45b622cac9d54ce80349a8db0fd85101e&get=NAME&for=county%3A%2A&in=state%3A51
URL ERROR, MAKE SURE IT IS CORRECTLY CONFIGURED
'str' object cannot be interpreted as an integer
'str' object cannot be interpreted as an integer
'str' object cannot be interpreted as an integer
https://api.census.gov/data/2022/acs/acs1/profile?key=9602e5c45b622cac9d54ce80349a8db0fd85101e&get=NAME&for=county%3A%2A&in=state%3A51
URL ERROR, MA

In [ ]:
# query = """
# SELECT * 
# FROM cities
# """

# counties = list(pd.read_sql_query(query, con=engine)['county'])

# final_df = pd.DataFrame()
# for county in counties:
#     print("Working on: ", county)
#     city_data_df = ca.get_data_for_city(city=None, years=(2009,2024), county=county)
#     final_df = pd.concat([final_df, city_data_df], ignore_index=True)


In [ ]:
# ca.upload_city_data_to_postgres(final_df, engine=engine)

In [ ]:
# # existing_data = pd.read_sql('SELECT city, year FROM city_data', engine)
# # new_rows = final_df[~final_df[['city', 'year']].isin(existing_data[['city', 'year']])]
# # final_df[['city', 'year']].isin(existing_data[['city', 'year']])
# query = """
# SELECT * FROM city_data
# """

# check_df = pd.read_sql_query(query, con=engine)
# check_df
# #rows_to_upload = final_df[]
# # final_df.to_sql("city_data", con=engine, index=False, chunksize=1000, if_exists="replace")

In [ ]:

# query = """
# SELECT * FROM city_data
# """

# df = pd.read_sql_query(query, con=engine)
# df

In [ ]:
# y = "2009"
# url = "https://api.census.gov/data/" + f"{y}/acs/acs1/profile"

# get_metrics = "".join(x + "," for x in ca.list_of_metrics.keys())
# get_metrics = get_metrics[:len(get_metrics)-1] # get rid of the last comma that I appended

# params = {"key": os.getenv("CENSUS_KEY"),
#                     "get": get_metrics,
#                     "for": f"county:810",
#                     "in": "state:51"}

# r = requests.get(url=url, params=params)
# values = list(json.loads(r.text))
# pd.DataFrame(values[1:], columns=values[0])

In [ ]:
# ca.get_all_cities()

In [ ]:
# from html.parser import HTMLParser
# #census_url = "https://api.census.gov/data"
# year = "2023"
# dataset = "acs"
# url = f"https://api.census.gov/data/{year}/acs/acs1/profile"


# """
#     # https://api.census.gov/data/2022/acs/acs1/profile/variables.html 

#     # ones worth using:

#     # economic
#     DP03_0093E = estimate for earnings for first year male workers 
#     DP03_0094E = estimate for earnings for first year female workers
#     DP03_0119PE = estimated percentage of people below the poverty line
#     DP04_0001E = estimated housing units
#     DP02_0002E = estimated number of married housing units
#     DP02_0016E = estimated average household size

#     DP03_0052E: estimated total households less than $10,000 income
#     DP03_0053E: estimated total households between $10-15k
#     DP03_0054E: estimated total households between $15-25k
#     DP03_0055E: 25-35k
#     DP03_0056E: 35-45k
#     DP03_0057E: 50-75k
#     DP03_0058E: 75-100k
#     DP03_0059E: 100-150k
#     DP03_0060E: 150-200k
#     DP03_0061E: > 200k

#     # relationships
#     DP02_0026E = men over 15 y/o who have not married
#     DP02_0028E = men over 15 y/o who married, but separated
#     DP02_0030E = men over 15 y/o who are divorced
    
#     # education (currently)
#     DP02_0054E = estimated population of kids in nursery school/preschool
#     DP02_0055E = estimated population of kids in kindergarten
#     DP02_0056E = estimated population of kids in grades 1-8
#     DP02_0057E = estimated population of kids in grades 9-12
#     DP02_0058E = estimated population of kids in college/grad school

#     # education attainment
#     DP02_0060PE = estimated population of 25 years and older who have less than 9th grade education
#     DP02_0061PE = estimated population with some high school education
#     DP02_0062PE = estimated population with high school diploma
#     DP02_0063PE = estimated population with some college, no diploma
#     DP02_0064PE = estimated population with associates degrees
#     DP02_0065PE  = estimated population with bachelor's degree
#     DP02_0066PE = estimated population with graduate degrees

#     # unemployement rate
#     DP03_0009PE

#     # industry
#     DP03_0033PE: agriculture, forestry, fishing, hunting, mining
#     DP03_0034PE: construction
#     DP03_0035PE: manufacturing
#     DP03_0036PE: wholesale trade
#     DP03_0037PE: retail trade
#     DP03_0038PE: transportation, warehousing, utilities
#     DP03_0039PE: information
#     DP03_0040PE: finance, insurance, real estate, rental, leasing
#     DP03_0041PE: professional, scientific, administrative, waste management
#     DP03_0042PE: educational services/health care/ social assistance
#     DP03_0043PE: arts, entertainment, recreation, accommodation, food services
#     DP03_0045PE: public administration

# """
# variables = "NAME,DP02_0060PE,DP02_0061PE,DP02_0062PE,DP02_0063PE,DP02_0064PE,DP02_0065PE,DP02_0066PE"
# print(url)
# params = {"key": os.getenv("CENSUS_KEY"),
#            "for": "county:*", 
#            "get": variables,
#            "in": "state:51"
#           }
# r = requests.get(url, params=params)
# values = list(json.loads(r.text))

# df = pd.DataFrame(values[1:], columns=values[0])
# # df = df.rename(columns={
# #     "DP02_0060PE":"<9th",
# #     "DP02_0061PE": "some high school",
# #     "DP02_0062PE": "high school",
# #     "DP02_0063PE": "some college",
# #     "DP02_0064PE": "associates",
# #     "DP02_0065PE": "bachelors",
# #     "DP02_0066PE": "graduate"
# # })
# df

In [ ]:
co.upload_data_to_postgres(climate_data=climate_df, engine=engine)

In [ ]:
query = """
SELECT *
FROM climate_data
"""

pd.read_sql_query(query, con=engine)

In [ ]:
cities = cen.get_all_cities()
cities = list(cities['city'])

city_to_station = {}

full_df = pd.DataFrame()
for c in cities:
    if c == "fauquier":
        c = "front royal"
    if c == "jamescity":
        c = "williamsburg"
    if c == "princewilliam":
        c = "manassas"
    if c == "spotsylvania":
        c = "fredericksburg"

    if c == "beach":
        c = "Virginia Beach"
    c = c + ", virginia"
    print("Working on:", c)
    station_id = cli.get_station_id(c)
    climate_df = cli.get_data_for_station_id(station_id=station_id, start_date="1940-01-01", end_date="2023-12-31")
    full_df = pd.concat([full_df, climate_df], ignore_index=True)

In [ ]:
station_id = cli.get_station_id("virginia beach, virginia")
climate_df = cli.get_data_for_station_id(station_id=station_id, start_date="1940-01-01", end_date="2023-12-31")
full_df = pd.concat([full_df, climate_df], ignore_index=True)

In [ ]:
full_df = full_df.rename(columns={
    "STATION": "id",
    "DATE": "date",
    "PRCP": "prcp",
    "TMAX": "tmax",
    "TMIN": "tmin"
})


cli.upload_data_to_postgres(full_df, engine=engine)

Uploading city climate data to the database
city_data table does not exist yet, creating now with first command
Finished uploading city data to database


In [10]:
query = """
SELECT *
FROM climate_data
WHERE id='USW00093736'
"""
pd.read_sql_query(query, con=engine)

,id,date,prcp,tmax,tmin
0,USW00093736,1955-09-14,NaN,77.0,47.0
1,USW00093736,1955-09-15,NaN,88.0,55.0
2,USW00093736,1955-09-16,NaN,92.0,64.0
3,USW00093736,1955-09-17,NaN,80.0,70.0
4,USW00093736,1955-09-18,NaN,81.0,67.0
...,...,...,...,...,...
24879,USW00093736,2023-12-27,0.28,55.0,50.0
24880,USW00093736,2023-12-28,0.00,61.0,41.0
24881,USW00093736,2023-12-29,0.02,50.0,36.0
24882,USW00093736,2023-12-30,0.00,51.0,34.0


In [ ]:
col_one = city_to_station.keys()
col_two = city_to_station.values()

new_dicts = {"city": col_one, "id": col_two}

city_id_df = pd.DataFrame.from_dict(new_dicts)

In [ ]:
cli.upload_cities_and_ids(city_id_df, engine=engine)

In [ ]:
lat, long = co.get_lat_long(location="Fredericksburg VA")
bb = co.get_bounding_box(latitude=lat, longitude=long, buffer = 10)
bb

In [ ]:
bounding_box = f"{bb[0]},{bb[1]},{bb[2]},{bb[3]}"
print(bounding_box)
params = {"dataset":"global-summary-of-the-month",
                  "bbox": bounding_box,
                  "dataTypes": "TMIN,TMAX,PRCP",
                  "startDate": "2022-01-01",
                  "endDate": "2022-02-01"}
headers = co.make_headers()
base = "https://www.ncei.noaa.gov/access/services/search/v1/data"
r = requests.get(url=base, params=params, headers=headers)
r.text

In [ ]:
city_to_station

In [ ]:
final_df = pd.DataFrame()
for station in city_to_station.values():
    co.get_data_for_station_id(station)